Titanic dataset

In [ ]:
import pandas as pd
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]
print(data[data.isna().any(axis =1)]) #проверка на Nan
data.head(5)

     PassengerId  Survived  Pclass                                      Name  \
0              1         0       3                   Braund, Mr. Owen Harris   
2              3         1       3                    Heikkinen, Miss. Laina   
4              5         0       3                  Allen, Mr. William Henry   
5              6         0       3                          Moran, Mr. James   
7              8         0       3            Palsson, Master. Gosta Leonard   
..           ...       ...     ...                                       ...   
884          885         0       3                    Sutehall, Mr. Henry Jr   
885          886         0       3      Rice, Mrs. William (Margaret Norton)   
886          887         0       2                     Montvila, Rev. Juozas   
888          889         0       3  Johnston, Miss. Catherine Helen "Carrie"   
890          891         0       3                       Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch        

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
def clean(data):
    data = data.drop(["PassengerId","Name","Cabin","Ticket"],axis = 1) #Удаляем не нужные столбцы
    colmns = ["Age", "Fare", "SibSp", "Parch"] 
    for col in colmns:
        # Вместе пропуска используется среднее заначение
        data[col].fillna(data[col].median(), inplace = True) 
    data.Embarked.fillna("U",inplace=True) # Заполняем пропуски и вмесесто пропуска U
    return data
    
data = clean(data)
test = clean(test)

In [ ]:
data.sample(5)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
476,0,2,male,34.0,1,0,21.0000,S
108,0,3,male,38.0,0,0,7.8958,S
207,1,3,male,26.0,0,0,18.7875,C
853,1,1,female,16.0,0,1,39.4000,S
507,1,1,male,28.0,0,0,26.5500,S


In [ ]:
from sklearn import preprocessing 
# Используем LableEncoder для преобразования в численные переменные
model = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked"]

# Для столбцов из "col" изсползуем кодирование
for i in cols:
    data[i] = model.fit_transform(data[i])
    test[i] = model.transform(test[i])
    print(model.classes_)
data.head(5)


['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

y = data["Survived"]
X = data.drop("Survived", axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Обучаем на train data
clf = LogisticRegression(random_state = 0, max_iter=1000).fit(X_train, y_train)

# Предсказываем
prd = clf.predict(X_val)

print("Accuracy = ",accuracy_score(y_val, prd))

Accuracy =  0.8100558659217877


In [ ]:
prd_test = clf.predict(test)

In [ ]:
#Используем DataFrame для звполнения таблицы предсказанными данными
df = pd.DataFrame({"PassengerId":test_ids.values,
                   "Survived":prd_test})


In [ ]:
#Создание csv файла
df.to_csv("submission.csv", index = False)